In [2]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from PIL import Image
import os
from pathlib import Path
from utils import *
from allResNets import *
from engine import *
from data_transforms import create_transform
from dataloaders import *
import cv2
from data_preparation import *
import torch
import glob
import pandas as pd

In [20]:
def find_classes(csv_file: str) -> List[str]:
    classes = []
    
    df = pd.read_csv(csv_file)

    for index, row in df.iterrows():
        labels = []

        if(row['Camera Alignment']):
            labels.append('Camera Alignment')

        if(row['Obstructed Camera']):
            labels.append('Obstructed Camera')

        if(row['Over lighting']):
            labels.append('Over lighting')

        if(row['Under lighting']):
            labels.append('Under lighting')

        if(row['Saturated']):
            labels.append('Saturated')

        classes.append(labels)
    
    return classes
    


In [21]:
class CustomImageFolderMultiLabel(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform = None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.image_paths = "Quality/dataset/" + self.data.iloc[:,-1] + "/" +  self.data.iloc[:,0] + ".png"
        self.labels = self.data.iloc[:,2:7]
        self.classes = find_classes(csv_file)

    def __getitem__(self, index):
        image_path = self.image_paths.iloc[index]
        label = self.labels.iloc[index]
        classes = self.classes[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label, classes
    
    def __len__(self):
        return len(self.image_paths)

In [4]:
import csv
from utils import *

change_to_disk()

# Open the input and output files
with open('Quality/data/new_data/all_data_wos_double.csv', 'r') as input_file, open('Quality/data/new_data/all_data_wos_double_wcls.csv', 'w', newline='') as output_file:
    # Create a CSV reader and writer objects
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    
    # Iterate through the rows in the input file
    for row in reader:
        buff = []
        # Process the row (e.g., filter out unwanted rows, transform data, etc.)
        # In this example, we simply write the row to the output file
        #if (row[2] == "1") or (row[3] == "1") or row[4] == "1" or row[5] == "1":
        if (row[2] == "1"):
            buff.append('Bad Camera Alignment')
        if(row[3] == "1"):
            buff.append('Obstructed Camera')
        if(row[4] == "1"):
            buff.append('Overlighting')
        if(row[4] == "2"):
            buff.append('Underlighting')
        
        row[1] = buff

        writer.writerow(row)

In [5]:
import csv
from utils import *

change_to_disk()

# Open the input and output files
with open('Quality/data/new_data/all_data_wos_double_wcls.csv', 'r') as input_file, open('Quality/data/new_data/all_data_wos_double_wcls_labeled.csv', 'w', newline='') as output_file:
    # Create a CSV reader and writer objects
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    
    # Iterate through the rows in the input file
    for row in reader:
        # Process the row (e.g., filter out unwanted rows, transform data, etc.)
        # In this example, we simply write the row to the output file
        if (row[2] == "1") or (row[3] == "1") or row[4] == "1" or row[5] == "1":
            writer.writerow(row)

In [ ]:
# Open the input and output files
with open('Quality/data/all_data_labeled_wos.csv', 'r') as input_file, open('Quality/data/double_data/all_data_wos.csv', 'w', newline='') as output_file:
    # Create a CSV reader and writer objects
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
    
    # Iterate through the rows in the input file
    for row in reader:
        # Process the row (e.g., filter out unwanted rows, transform data, etc.)
        # In this example, we simply write the row to the output file
        if row[0].startswith("Qa_align_1_0_"):
            writer.writerow(row)

In [400]:
import csv
from utils import *
change_to_disk()

# Open the input and output files
with open('Quality/data/double_data/data_labeled_wos_joined_double.csv', 'r') as input_file, open('Quality/data/placeholder.csv','w', newline='') as placeholder_file:
    # Create a CSV reader and writer objects
    reader = csv.reader(input_file)
    
    placeholder_writer = csv.writer(placeholder_file)
    counter = 0

    
    # Iterate through the rows in the input file
    for row in reader:
        # Process the row (e.g., filter out unwanted rows, transform data, etc.)
        # In this example, we simply write the row to the output file
        if row[5].startswith("4_Under_lighting") and row[0].startswith("Qa_Ulight_5_1"):
            counter = counter+1
            placeholder_writer.writerow(row)

In [397]:
with open('Quality/data/placeholder.csv','r') as placeholder_file, open('Quality/data/double_data/train_data_double.csv', 'a', newline='') as train_file, open('Quality/data/double_data/test_validation_data_double.csv','a',newline='') as test_val_file:
    reader = csv.reader(placeholder_file)
    train_writer = csv.writer(train_file)
    test_val_writer = csv.writer(test_val_file)
    split_count = 0

    for row in reader:
        if 0.7*counter > split_count:
            train_writer.writerow(row)
        elif 0.8*counter < split_count:
            test_val_writer.writerow(row)
        
        split_count += 1

In [ ]:
##Split in classes with 1k per class / whole class

classes = ["1_Camera_alignment", "2_Obstructed_Cameras", "3_Over_lighting", "4_Under_lighting", "5_Saturated"]

with open('Quality/data/train_data.csv','r') as train_data_file, open('Quality/data/test_validation_data.csv', 'r') as test_val_data_file, open('Quality/data/split_1k_train_data.csv','w',newline='') as split_1k_train_data_file, open('Quality/data/split_1k_test_data.csv','w',newline='') as split_1k_test_data_file, open('Quality/data/split_1k_validation_data.csv','w',newline='') as split_1k_validation_data_file:
    reader_train = csv.reader(train_data_file)
    reader_test_validation = csv.reader(test_val_data_file)

    row_count_test_val = sum(1 for row in reader_test_validation)
    row_count_train = sum(1 for row in reader_train)


In [ ]:
with open('Quality/data/train_data.csv','r') as train_data_file, open('Quality/data/test_validation_data.csv', 'r') as test_val_data_file, open('Quality/data/split_1k_train_data.csv','w',newline='') as split_1k_train_data_file, open('Quality/data/split_1k_test_data.csv','w',newline='') as split_1k_test_data_file, open('Quality/data/split_1k_validation_data.csv','w',newline='') as split_1k_validation_data_file:
    reader_train = csv.reader(train_data_file)
    reader_test_validation = csv.reader(test_val_data_file)

    split_1k_train_writer = csv.writer(split_1k_train_data_file)
    split_1k_validation_writer = csv.writer(split_1k_validation_data_file)
    split_1k_test_writer = csv.writer(split_1k_test_data_file)

    num_files = 5000
    array_range_train = range(0,row_count_train)
    array_range_validation = range(0,row_count_test_val)
    array_range_test = range(0,row_count_test_val)
    
    array_train = random.sample(array_range_train, int(num_files*0.8))
    array_validation = random.sample(array_range_validation, int(num_files*0.1))
    array_test = random.sample(array_range_test,int(num_files*0.1))

    for i,row in enumerate(reader_test_validation):
        if(i in array_test):
            split_1k_test_writer.writerow(row)
        elif(i in array_validation):
            split_1k_validation_writer.writerow(row)

    for i,row in enumerate(reader_train):
        if(i in array_train):
            split_1k_train_writer.writerow(row)

Separação por classes para depois conseguir extrair o numero que é necessário por classe


In [403]:
with open('Quality/data/double_data/test_validation_data_double.csv','r') as train_data_file, open('Quality/data/test_validation_by_classes/1_Camera_alignment.csv', 'w', newline='') as camera_alignment_file, open('Quality/data/test_validation_by_classes/2_Obstructed_cameras.csv', 'w', newline='') as obstructed_cameras_file, open('Quality/data/test_validation_by_classes/3_Over_lighting.csv', 'w', newline='') as over_lighting_file, open('Quality/data/test_validation_by_classes/4_Under_lighting.csv', 'w', newline='') as under_lighting_file:
    reader_train = csv.reader(train_data_file)

    writer_camera_alignment = csv.writer(camera_alignment_file)
    writer_obstructed_cameras = csv.writer(obstructed_cameras_file)
    writer_over_lighting = csv.writer(over_lighting_file)
    writer_under_lighting = csv.writer(under_lighting_file)

    for row in reader_train:
        if row[5].startswith("1_Camera_alignment"):
            writer_camera_alignment.writerow(row)
        elif row[5].startswith("2_Obstructed_cameras"):
            writer_obstructed_cameras.writerow(row)
        elif row[5].startswith("3_Over_lighting"):
            writer_over_lighting.writerow(row)
        elif row[5].startswith("4_Under_lighting"):
            writer_under_lighting.writerow(row)


Tentar agora equilibrar as classes

Treino

In [195]:
with open('Quality/data/train_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/train_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/train_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/train_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file:
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)


    row_count_camera_alignment = sum(1 for row in camera_alignment_reader)
    row_count_obstructed_cameras = sum(1 for row in obstructed_cameras_reader)
    row_count_over_lighting = sum(1 for row in over_lighting_reader)
    row_count_under_lighting = sum(1 for row in under_lighting_reader)

In [196]:
with open('Quality/data/split_5k_train_data_single.csv','a', newline='') as train_data_file, open('Quality/data/train_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/train_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/train_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/train_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file:
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)

    img_per_class_train = 800

    train_writer = csv.writer(train_data_file)

    array_range_camera_alignment = range(0,row_count_camera_alignment)
    array_range_obstructed_cameras = range(0,row_count_obstructed_cameras)
    array_range_over_lighting = range(0,row_count_over_lighting)
    array_range_under_lighting = range(0, row_count_under_lighting)

    array_camera_alignment = random.sample(array_range_camera_alignment, img_per_class_train)
    array_obstructed_cameras = random.sample(array_range_obstructed_cameras, img_per_class_train)
    array_over_lighting = random.sample(array_range_over_lighting, img_per_class_train)
    array_under_lighting = random.sample(array_range_under_lighting, img_per_class_train)

    for i,row in enumerate(camera_alignment_reader):
        if i in array_camera_alignment:
            train_writer.writerow(row)

    for i,row in enumerate(obstructed_cameras_reader):
        if i in array_obstructed_cameras:
            train_writer.writerow(row)

    for i,row in enumerate(over_lighting_reader):
        if i in array_over_lighting:
            train_writer.writerow(row)
    
    for i,row in enumerate(under_lighting_reader):
        if i in array_under_lighting:
            train_writer.writerow(row)


ValueError: Sample larger than population or is negative

Teste e validação

In [193]:
with open('Quality/data/test_validation_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/test_validation_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/test_validation_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/test_validation_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file:
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)


    row_count_camera_alignment = sum(1 for row in camera_alignment_reader)
    row_count_obstructed_cameras = sum(1 for row in obstructed_cameras_reader)
    row_count_over_lighting = sum(1 for row in over_lighting_reader)
    row_count_under_lighting = sum(1 for row in under_lighting_reader)

In [194]:
with open('Quality/data/split_5k_test_data.csv','a', newline='') as test_data_file, open('Quality/data/split_5k_validation_data.csv','a', newline='') as validation_data_file , open('Quality/data/test_validation_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/test_validation_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/test_validation_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/test_validation_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file :
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)

    img_per_class_val_test = 500

    test_writer = csv.writer(test_data_file)
    validation_writer = csv.writer(validation_data_file)

    array_range_camera_alignment_test = range(0,row_count_camera_alignment)
    array_range_obstructed_cameras_test = range(0,row_count_obstructed_cameras)
    array_range_over_lighting_test = range(0,row_count_over_lighting)
    array_range_under_lighting_test = range(0, row_count_under_lighting)
    
    array_camera_alignment_test = random.sample(array_range_camera_alignment_test, img_per_class_val_test)
    array_obstructed_cameras_test = random.sample(array_range_obstructed_cameras_test, img_per_class_val_test)
    array_over_lighting_test = random.sample(array_range_over_lighting_test, img_per_class_val_test)
    array_under_lighting_test = random.sample(array_range_under_lighting_test, img_per_class_val_test)

    array_range_camera_alignment_validation = range(0,row_count_camera_alignment)
    array_range_obstructed_cameras_validation = range(0,row_count_obstructed_cameras)
    array_range_over_lighting_validation = range(0,row_count_over_lighting)
    array_range_under_lighting_validation = range(0, row_count_under_lighting)
    
    array_camera_alignment_validation = random.sample(array_range_camera_alignment_validation, img_per_class_val_test)
    array_obstructed_cameras_validation = random.sample(array_range_obstructed_cameras_validation, img_per_class_val_test)
    array_over_lighting_validation = random.sample(array_range_over_lighting_validation, img_per_class_val_test)
    array_under_lighting_validation = random.sample(array_range_under_lighting_validation, img_per_class_val_test)


    for i,row in enumerate(camera_alignment_reader):
        if i in array_camera_alignment_test:
            test_writer.writerow(row)
        if i in array_camera_alignment_validation:
            validation_writer.writerow(row)

    for i,row in enumerate(obstructed_cameras_reader):
        if i in array_obstructed_cameras_test:
            test_writer.writerow(row)
        if i in array_obstructed_cameras_validation:
            validation_writer.writerow(row)

    for i,row in enumerate(over_lighting_reader):
        if i in array_over_lighting_test:
            test_writer.writerow(row)
        if i in array_over_lighting_validation:
            validation_writer.writerow(row)
    
    for i,row in enumerate(under_lighting_reader):
        if i in array_under_lighting_test:
            test_writer.writerow(row)
        if i in array_under_lighting_validation:
            validation_writer.writerow(row)



Modificado para 3 classes

Treino

In [404]:
with open('Quality/data/train_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/train_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/train_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/train_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file:
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)


    row_count_camera_alignment = sum(1 for row in camera_alignment_reader)
    row_count_obstructed_cameras = sum(1 for row in obstructed_cameras_reader)
    row_count_over_lighting = sum(1 for row in over_lighting_reader)
    row_count_under_lighting = sum(1 for row in under_lighting_reader)
    row_count_lighting_issues = row_count_over_lighting + row_count_under_lighting

In [405]:
with open('Quality/data/split_1k_train_data_double.csv','a', newline='') as train_data_file, open('Quality/data/train_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/train_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/train_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/train_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file:
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)

    img_per_class_train = 800

    train_writer = csv.writer(train_data_file)

    array_range_camera_alignment = range(0,row_count_camera_alignment)
    array_range_obstructed_cameras = range(0,row_count_obstructed_cameras)
    array_range_over_lighting = range(0,row_count_over_lighting)
    array_range_under_lighting = range(0, row_count_under_lighting)

    array_camera_alignment = random.sample(array_range_camera_alignment, img_per_class_train)
    array_obstructed_cameras = random.sample(array_range_obstructed_cameras, img_per_class_train)
    array_over_lighting = random.sample(array_range_over_lighting, int(img_per_class_train/2))
    array_under_lighting = random.sample(array_range_under_lighting, int(img_per_class_train/2))

    for i,row in enumerate(camera_alignment_reader):
        if i in array_camera_alignment:
            train_writer.writerow(row)

    for i,row in enumerate(obstructed_cameras_reader):
        if i in array_obstructed_cameras:
            train_writer.writerow(row)

    for i,row in enumerate(over_lighting_reader):
        if i in array_over_lighting:
            train_writer.writerow(row)
    
    for i,row in enumerate(under_lighting_reader):
        if i in array_under_lighting:
            train_writer.writerow(row)

In [406]:
with open('Quality/data/test_validation_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/test_validation_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/test_validation_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/test_validation_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file:
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)


    row_count_camera_alignment = sum(1 for row in camera_alignment_reader)
    row_count_obstructed_cameras = sum(1 for row in obstructed_cameras_reader)
    row_count_over_lighting = sum(1 for row in over_lighting_reader)
    row_count_under_lighting = sum(1 for row in under_lighting_reader)
    row_count_lighting_issues = row_count_over_lighting + row_count_under_lighting

In [407]:
with open('Quality/data/split_1k_test_data_double.csv','a', newline='') as test_data_file, open('Quality/data/split_1k_validation_data_double.csv','a', newline='') as validation_data_file , open('Quality/data/test_validation_by_classes/1_Camera_alignment.csv', 'r') as camera_alignment_file, open('Quality/data/test_validation_by_classes/2_Obstructed_cameras.csv', 'r') as obstructed_cameras_file, open('Quality/data/test_validation_by_classes/3_Over_lighting.csv', 'r') as over_lighting_file, open('Quality/data/test_validation_by_classes/4_Under_lighting.csv', 'r') as under_lighting_file :
    camera_alignment_reader = csv.reader(camera_alignment_file)
    obstructed_cameras_reader = csv.reader(obstructed_cameras_file)
    over_lighting_reader = csv.reader(over_lighting_file)
    under_lighting_reader = csv.reader(under_lighting_file)

    img_per_class_val_test = 120

    test_writer = csv.writer(test_data_file)
    validation_writer = csv.writer(validation_data_file)

    array_range_camera_alignment_test = range(0,row_count_camera_alignment)
    array_range_obstructed_cameras_test = range(0,row_count_obstructed_cameras)
    array_range_over_lighting_test = range(0,row_count_over_lighting)
    array_range_under_lighting_test = range(0, row_count_under_lighting)
    
    array_camera_alignment_test = random.sample(array_range_camera_alignment_test, img_per_class_val_test)
    array_obstructed_cameras_test = random.sample(array_range_obstructed_cameras_test, img_per_class_val_test)
    array_over_lighting_test = random.sample(array_range_over_lighting_test, int(img_per_class_val_test/2))
    array_under_lighting_test = random.sample(array_range_under_lighting_test, int(img_per_class_val_test/2))

    array_range_camera_alignment_validation = range(0,row_count_camera_alignment)
    array_range_obstructed_cameras_validation = range(0,row_count_obstructed_cameras)
    array_range_over_lighting_validation = range(0,row_count_over_lighting)
    array_range_under_lighting_validation = range(0, row_count_under_lighting)
    
    array_camera_alignment_validation = random.sample(array_range_camera_alignment_validation, img_per_class_val_test)
    array_obstructed_cameras_validation = random.sample(array_range_obstructed_cameras_validation, img_per_class_val_test)
    array_over_lighting_validation = random.sample(array_range_over_lighting_validation, int(img_per_class_val_test/2))
    array_under_lighting_validation = random.sample(array_range_under_lighting_validation, int(img_per_class_val_test/2))


    for i,row in enumerate(camera_alignment_reader):
        if i in array_camera_alignment_test:
            test_writer.writerow(row)
        if i in array_camera_alignment_validation:
            validation_writer.writerow(row)

    for i,row in enumerate(obstructed_cameras_reader):
        if i in array_obstructed_cameras_test:
            test_writer.writerow(row)
        if i in array_obstructed_cameras_validation:
            validation_writer.writerow(row)

    for i,row in enumerate(over_lighting_reader):
        if i in array_over_lighting_test:
            test_writer.writerow(row)
        if i in array_over_lighting_validation:
            validation_writer.writerow(row)
    
    for i,row in enumerate(under_lighting_reader):
        if i in array_under_lighting_test:
            test_writer.writerow(row)
        if i in array_under_lighting_validation:
            validation_writer.writerow(row)

In [3]:
#Change disk directory
base_path = Path("E:/Users/joaor/NSP/2021")
if(Path().cwd() != Path(r"E:\Users\joaor\NSP\2021")):
    os.chdir(base_path)

video_name = "Video_10_Poor"

videos_folder = Path("Visibility/" + video_name)
videos = glob.glob(str(videos_folder) + "/*.mp4")

save_folder = Path("Visibility_frames/"+video_name)
check_dir(save_folder)
save_folder = str(save_folder / video_name)

print(videos)

for i,video in enumerate(videos):
    convert_mp4_to_png(video  ,save_folder+"_"+str(i))

Visibility_frames\Video_10_Poor does not exist, creating one...
['Visibility\\Video_10_Poor\\20210329114717566@Block209F_HD1_PORT.mp4', 'Visibility\\Video_10_Poor\\20210329114717590@Block209F_HD2_CENTRE.mp4', 'Visibility\\Video_10_Poor\\20210329114717609@Block209F_HD3_STBD.mp4']
Frame:0 | Success:True
Frame:500 | Success:True
Frame:1000 | Success:True
Frame:1500 | Success:True
Frame:2000 | Success:True
Frame:2500 | Success:True
Frame:3000 | Success:True
Frame:3500 | Success:True
Frame:4000 | Success:True
Frame:4500 | Success:True
Frame:5000 | Success:True
Frame:5500 | Success:True
Frame:6000 | Success:True
Frame:6500 | Success:True
Frame:7000 | Success:True
Frame:7500 | Success:True
Frame:8000 | Success:True
Frame:8500 | Success:True
Frame:9000 | Success:True
Frame:9500 | Success:True
Frame:10000 | Success:True
Frame:10500 | Success:True
Frame:11000 | Success:True
Frame:11500 | Success:True
Frame:12000 | Success:True
Frame:12500 | Success:True
Frame:13000 | Success:True
Frame:13500 | S